In [2]:
from secret import SECRET
import networkx as nx
import re
import json
import random
from urllib.parse import quote
import requests
import folium as f
import pprint

BASE_URL = "https://maps.googleapis.com/maps/api/directions/json?"
URL_END = "&mode=bicycling&key=" + SECRET
WAYPOINT_MAX = 23 # In case we ever use waypoints

g_pickle = "data/02_cambridge.gpickle"
g = nx.read_gpickle(g_pickle)
test_points = random.sample(list(g.nodes().data()), 2)

In [3]:
def decode_polyline(polyline_str):
    '''Pass a Google Maps encoded polyline string; returns list of lat/lon pairs
       WRITTEN BY mgd722
    '''
    index, lat, lng = 0, 0, 0
    coordinates = []
    changes = {'latitude': 0, 'longitude': 0}

    # Coordinates have variable length when encoded, so just keep
    # track of whether we've hit the end of the string. In each
    # while loop iteration, a single coordinate is decoded.
    while index < len(polyline_str):
        # Gather lat/lon changes, store them in a dictionary to apply them later
        for unit in ['latitude', 'longitude']: 
            shift, result = 0, 0

            while True:
                byte = ord(polyline_str[index]) - 63
                index += 1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break

            if (result & 1):
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)

        lat += changes['latitude']
        lng += changes['longitude']

        coordinates.append((lat / 100000.0, lng / 100000.0))

    return coordinates

In [4]:
def generate_url(origin, dest):
    origin_param = "&" + "origin=" +quote((str(origin['lat']) + ',' + str(origin['lon'])))
    dest_param = "&" + "destination=" + quote((str(dest['lat']) + ',' + str(dest['lon'])))

    url = BASE_URL + origin_param + dest_param + URL_END
    return url
url = generate_url(origin = test_points[0][1], dest = test_points[1][1])

In [8]:
# route_data = requests.get(url).json()

# steps = route_data['routes'][0]['legs'][0]['steps']

# needed_intermediary_signs_num = (len(steps) - 1)
# needed_intermediary_signs_num

route_data

{'error_message': 'The provided API key is expired.',
 'routes': [],
 'status': 'REQUEST_DENIED'}

In [ ]:
def draw_map():
    m = f.Map(location = [42.3736,-71.109977], zoom_start=14)

    for node_data in test_points:
        node_id = node_data[1]["id"]
        lon,lat = node_data[1]['lon'], node_data[1]['lat'] 

        # CSS stuff
        size = 25
        color = "magenta"

        f.Marker(location=[lat,lon],
                  # Set icon to DivIcon to with conditional style formatting to reference the random temp value.
                  icon=f.DivIcon(html=f"""<div style="border:1px solid {color}; text-align: center; vertical-align: middle; line-height: {size}px; background-color: white; width: {size}px; height: {size}px; border-radius: {size}px; font-family: sans-serif; color: {color};">{45}</div>""")
                 ).add_to(m)
    return m

In [6]:
r = lambda: random.randint(0,255)

m = draw_map()
for step in steps:
    encoded_polyline = step["polyline"]["points"]
    polyline = decode_polyline(encoded_polyline)
#     m.add_child(f.PolyLine(locations=polyline, line_opacity=1, color='#%02X%02X%02X' % (r(),r(),r())))
    m.add_child(f.PolyLine(locations=polyline, opacity=.6, color="magenta"))

m

NameError: name 'draw_map' is not defined